In [1]:
import os
import cv2
import torch
import random
import numpy as np
from torch import nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from torchvision import transforms


In [13]:
class EyeDataset(Dataset):
    def __init__(self, folder_path, split='train', val_split=0.2, img_size=(86, 86)):
        self.data = []
        self.labels = []
        self.img_size = img_size
        self.label_map = {'Closed_Eyes': 0, 'Open_Eyes': 1}

        for label in self.label_map:
            path = os.path.join(folder_path, label)
            for img_name in os.listdir(path):
                img_path = os.path.join(path, img_name)
                img = cv2.imread(img_path)
                if img is None:
                    continue
                img = cv2.resize(img, self.img_size)
                img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                self.data.append(img)
                self.labels.append(self.label_map[label])

        # Split
        X_train, X_val, y_train, y_val = train_test_split(
            self.data, self.labels, test_size=val_split, stratify=self.labels, random_state=42
        )

        if split == 'train':
            self.data = X_train
            self.labels = y_train
        else:
            self.data = X_val
            self.labels = y_val

        self.transform = transforms.Compose([
            transforms.ToTensor(),               # Converts to [0, 1] tensor
            transforms.Normalize([0.5], [0.5])  # Normalize to [-1, 1]
        ])

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img = self.data[idx]
        label = self.labels[idx]
        img = self.transform(img)
        return img, torch.tensor(label, dtype=torch.long)


In [20]:
class EyeCNN(nn.Module):
    def __init__(self):
        super(EyeCNN, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=3, padding=1),  # [3,86,86] → [16,86,86]
            nn.ReLU(),
            nn.MaxPool2d(2, 2),                         # → [16,43,43]

            nn.Conv2d(16, 32, kernel_size=3, padding=1),# → [32,43,43]
            nn.ReLU(),
            nn.MaxPool2d(2, 2),                         # → [32,21,21]

            nn.Conv2d(32, 64, kernel_size=3, padding=1),# → [64,21,21]
            nn.ReLU(),
            nn.MaxPool2d(2, 2)                          # → [64,10,10]
        )

        self.fc = nn.Sequential(
            nn.Flatten(),                               # → [6400]
            nn.Linear(64 * 10 * 10, 128),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, 2)                           # Output: open or closed
        )

    def forward(self, x):
        x = self.conv(x)
        x = self.fc(x)
        return x


In [21]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load dataset
train_ds = EyeDataset("/kaggle/input/mrl-dataset/train", split='train')
val_ds   = EyeDataset("/kaggle/input/mrl-dataset/train", split='val')

train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
val_loader   = DataLoader(val_ds, batch_size=32)

model = EyeCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


In [22]:
for epoch in range(10):
    model.train()
    train_loss, train_correct = 0, 0

    for imgs, labels in train_loader:
        imgs, labels = imgs.to(device), labels.to(device)

        outputs = model(imgs)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        train_correct += (outputs.argmax(1) == labels).sum().item()

    acc = train_correct / len(train_ds)
    print(f"Epoch {epoch+1} - Train Loss: {train_loss:.4f}, Accuracy: {acc:.4f}")

    # Validation
    model.eval()
    val_correct = 0
    with torch.no_grad():
        for imgs, labels in val_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            val_correct += (outputs.argmax(1) == labels).sum().item()

    val_acc = val_correct / len(val_ds)
    print(f"Validation Accuracy: {val_acc:.4f}\n")

torch.save(model.state_dict(), "eye_cnn_model.pth")


Epoch 1 - Train Loss: 18.8398, Accuracy: 0.9116
Validation Accuracy: 0.9762

Epoch 2 - Train Loss: 4.3594, Accuracy: 0.9891
Validation Accuracy: 0.9825

Epoch 3 - Train Loss: 2.3838, Accuracy: 0.9938
Validation Accuracy: 0.9900

Epoch 4 - Train Loss: 2.6677, Accuracy: 0.9919
Validation Accuracy: 0.9938

Epoch 5 - Train Loss: 1.6122, Accuracy: 0.9962
Validation Accuracy: 0.9950

Epoch 6 - Train Loss: 0.6944, Accuracy: 0.9978
Validation Accuracy: 1.0000

Epoch 7 - Train Loss: 0.4920, Accuracy: 0.9988
Validation Accuracy: 1.0000

Epoch 8 - Train Loss: 0.3097, Accuracy: 0.9991
Validation Accuracy: 1.0000

Epoch 9 - Train Loss: 0.4267, Accuracy: 0.9988
Validation Accuracy: 1.0000

Epoch 10 - Train Loss: 0.2988, Accuracy: 0.9988
Validation Accuracy: 1.0000

